# 4 Graphical Pipelines

In the previous notebook, we considered sequential pipelines.

In [1]:
import warnings

warnings.filterwarnings("ignore")

## 4.1 What are Graphical Pipelines?
Recap sequential pipelines:

<img src="img/sequential_pipeline.png" width=900  style="background-color:white; padding:5px" />

Many tasks are non-sequential. To solve this two possibilities exist:
1. Nesting Sequential Pipelines.
2. Using Graphical Pipelines.


Generalised Graphical Pipeline in sktime:

* Graphical means that different steps may share the same predecessor or provide their outputs to the same successor (the dataflows can branch and merge).

<img src="img/graphical_pipeline.png" width=900  style="background-color:white; padding:5px" />


* Generalised means that the pipeline can be used for multiple tasks (e.g. forecasting, classification, ...).


**Note**

The graphical pipeline is still experimental. 
Thus, usual risk with bleeding edge features. 
However, we would be happy to get feedback on the graphical pipeline.


#### Forecasting Use-Case for Graphical Pipelines


The input of forecasters depends on the output of other forecasters, which same the same input.
* Forecaster could use the same preprocessing (branching of data flow)
* Forecaster could use outputs of multiple predeccessors (merging of data flow)

<img src="img/graphical_pipeline_example.png" width=900  style="background-color:white; padding:5px" />


### Credits
The graphical pipeline was first developed by pyWATTS [1] and was then adapted for sktime. The original implementation can be found [pyWATTS](https://github.com/KIT-IAI/pyWATTS). pyWATTS is a open source library developed at the Institute of Applied Informatics and Automation at the KIT and funded by HelmholtzAI.

> [1] Heidrich, Benedikt, et al. "pyWATTS: Python workflow automation tool for time series." arXiv preprint arXiv:2106.10157 (2021).

<img src="img/kit.png" height=60  style="background-color:white; padding:5px" /> 
<p></p>

<img src="img/helmholtz.png" width=900  style="background-color:white; padding:5px" />


Two ways to create graphical pipelines:

1. Pass all steps to the pipeline during initialisation as for the sequential pipeline.

In [2]:
from sktime.pipeline.pipeline import Pipeline
from sktime.transformations.series.difference import Differencer

differencer = Differencer()

pipe = Pipeline(
    [
        {"skobject": differencer, "name": "differencer", "edges": {"X": "y"}},
        #...,
    ]
)

**Note** if you add the same skobject instance multiple times, the graphical pipelines tracks the identity of these skobjects.

Alternatively, the pipeline can be also created using `add_step`

In [3]:
from sktime.pipeline.pipeline import Pipeline
from sktime.transformations.series.difference import Differencer

pipe = Pipeline()
differencer = Differencer()

pipe = pipe.add_step(
    differencer, "differencer", edges={"X": "y"}
)
#...

#### Summary of the arguments:
The `add_step`'s parameter or key of the dicts in the step list during initialisation are:

* skobject: The sktime object added to the pipeline
* name: The name of the step
* edges: The keys of the dictionary indicate the input of the skobject (X or y), and the values are the names of the steps that should be connected to the input argument. Note subsetting using `__` and feature union via lists are supported.
* method: The skobject's method that should be called. If not provided, the default method would be inferred based on the added skobject. This parameter is used for the inverse_transform method. Optional.
* kwargs: Additional keyword arguments passed to the sktime object. Optional.

#### Take Away Messsage:
* Two ways to construct graphical pipeline
    * Provide all information during initialisation
    * Add each step separetely using `add_step` method

### 4.2.2 A more complex example
The considered use-case is to forecast the inflation using forecasts of the real gross domestic product, real disposable personal income, and the unemployment rate. Furthermore the unemployment rate is forecasted using the same features except the unemployment rate itself.

<img src="img/graphical_pipeline_example.png" width=900 style="background-color:white; padding:5px" />


The data is taken from the macrodata dataset from the statsmodels package.

**Note** We stick with the `add_step` in the following.


Create Graphical Pipeline Instance

In [4]:
from sktime.pipeline.pipeline import Pipeline

pipe = Pipeline()

<img src="img/graphical_pipeline_example.png" width=900 style="background-color:white; padding:5px" />

Add Preprocessing

In [5]:

from sklearn.preprocessing import StandardScaler

from sktime.transformations.series.adapt import TabularToSeriesAdaptor
from sktime.transformations.series.detrend import Deseasonalizer

pipe = pipe.add_step(
    TabularToSeriesAdaptor(StandardScaler()), name="scaler", edges={"X": "X__unemp"}
)
pipe = pipe.add_step(
    Deseasonalizer(sp=4), name="deseasonalizer", edges={"X": "X__realgdp_realdpi"}
)

<img src="img/graphical_pipeline_example.png" width=900 style="background-color:white; padding:5px" />

Add forecastesr for GDP and DPI

In [6]:
from sklearn.linear_model import Ridge
from sktime.forecasting.compose import make_reduction

pipe = pipe.add_step(
    make_reduction(Ridge(), windows_identical=False, window_length=5),
    name="forecaster_gdp",
    edges={"y": "deseasonalizer__realgdp"},
)

pipe = pipe.add_step(
    make_reduction(Ridge(), windows_identical=False, window_length=5),
    name="forecaster_dpi",
    edges={"y": "deseasonalizer__realdpi"},
)

<img src="img/graphical_pipeline_example.png" width=900 style="background-color:white; padding:5px" />

Add Forecaster for unemployment rate that depends on forecasts of GDP and DPI

In [7]:
pipe = pipe.add_step(
    make_reduction(Ridge(), windows_identical=False, window_length=5),
    name="forecaster_unemp",
    edges={
        "y": "scaler__unemp",
        "X": [
            "forecaster_gdp",
            "forecaster_dpi",
        ],
    },
)

<img src="img/graphical_pipeline_example.png" width=900 style="background-color:white; padding:5px" />

Add forecaster for the inflation that depends on forecasted DPI and unemployment rate

In [8]:
pipe = pipe.add_step(
    make_reduction(Ridge(), windows_identical=False, window_length=5),
    name="forecaster_inflation",
    edges={"X": ["forecaster_dpi", "forecaster_unemp"], "y": "y"},
)

Load data and split them into train and test

In [9]:
from sktime.datasets import load_macroeconomic
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.model_selection import (
    temporal_train_test_split,
)

data = load_macroeconomic()

X = data[["realgdp", "realdpi", "unemp"]]
y = data[["infl"]]
fh = ForecastingHorizon([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

y_train, y_test, X_train, X_test = temporal_train_test_split(y, X=X, fh=fh)
X_train

,realgdp,realdpi,unemp
Period,,,
1959Q1,2710.349,1886.9,5.8
1959Q2,2778.801,1919.7,5.1
1959Q3,2775.488,1916.4,5.3
1959Q4,2785.204,1931.3,5.6
1960Q1,2847.699,1955.5,5.2
...,...,...,...
2005Q3,12683.153,9308.0,5.0
2005Q4,12748.699,9358.7,4.9
2006Q1,12915.938,9533.8,4.7


In [10]:
pipe.fit(y=y_train, X=X_train, fh=fh)
result = pipe.predict(X=None, fh=y_test.index)
result

,infl
Period,
2006Q4,3.090428
2007Q1,1.676421
2007Q2,0.219586
2007Q3,1.570087
2007Q4,0.350137
2008Q1,0.438966
2008Q2,0.615457
2008Q3,0.119022
2008Q4,0.257887


### 4.2.3 How does the graphical pipeline compare to the sequential pipeline?

Let us try to implement a simplified version of the above example using sequential pipelines with nesting.

<img src="img/graphical_pipeline_simplified.png" width=900  style="background-color:white; padding:5px" />


Create sequential pipelines for
* forecasting the GDP
* forecasting DPI,
* and unemployment rate.
* ColunmEnsembleForecaster to combine the forecasts of the GDP, DPI, UNEMP (Union of forecasts).
* Create the inflation forecaster using the ColumnEnsembleForecaster inside of a ForecastX.

(Details in advanced pipeline notebook of pydata Prague 2023)


In [11]:
from sktime.forecasting.compose import ColumnEnsembleForecaster, ForecastX
from sktime.transformations.series.subset import ColumnSelect

forecasting_pipeline_gdp = (
    ColumnSelect(["realgdp"])  # To train the forecaster only on the realgdp column
    * Deseasonalizer()
    * make_reduction(Ridge(), windows_identical=False, window_length=5)
)
forecasting_pipeline_dpi = (
    ColumnSelect(["realdpi"])
    * Deseasonalizer()
    * make_reduction(Ridge(), windows_identical=False, window_length=5)
)

forecasting_pipeline_unemp = (
    ColumnSelect(["unemp"])
    * Deseasonalizer()
    * make_reduction(Ridge(), windows_identical=False, window_length=5)
)

input_inflation_forecast = ColumnEnsembleForecaster(
    [
        ("realdpi", forecasting_pipeline_dpi, "realdpi"),
        ("realgdp", forecasting_pipeline_gdp, "realgdp"),
        ("unemp", forecasting_pipeline_unemp, "unemp"),
    ]
)

inflation_forecast = ForecastX(
    make_reduction(Ridge(), windows_identical=False, window_length=5),
    input_inflation_forecast,
)



In [12]:
inflation_forecast.fit(y=y_train, X=X_train, fh=fh)

inflation_forecast.predict()

,infl
2006Q4,3.979318
2007Q1,2.347512
2007Q2,1.443598
2007Q3,3.914533
2007Q4,2.533117
2008Q1,3.278010
2008Q2,3.861517
2008Q3,3.487510
2008Q4,4.195074
2009Q1,4.294984


In [13]:
list(inflation_forecast.get_params(True).keys())

['behaviour',
 'columns',
 'fh_X',
 'fit_behaviour',
 'forecaster_X',
 'forecaster_y',
 'forecaster_X__forecasters',
 'forecaster_X__realdpi',
 'forecaster_X__realgdp',
 'forecaster_X__unemp',
 'forecaster_X__realdpi__steps',
 'forecaster_X__realdpi__ColumnSelect',
 'forecaster_X__realdpi__Deseasonalizer',
 'forecaster_X__realdpi__RecursiveTabularRegressionForecaster',
 'forecaster_X__realdpi__ColumnSelect__columns',
 'forecaster_X__realdpi__ColumnSelect__index_treatment',
 'forecaster_X__realdpi__ColumnSelect__integer_treatment',
 'forecaster_X__realdpi__Deseasonalizer__model',
 'forecaster_X__realdpi__Deseasonalizer__sp',
 'forecaster_X__realdpi__RecursiveTabularRegressionForecaster__estimator',
 'forecaster_X__realdpi__RecursiveTabularRegressionForecaster__pooling',
 'forecaster_X__realdpi__RecursiveTabularRegressionForecaster__transformers',
 'forecaster_X__realdpi__RecursiveTabularRegressionForecaster__window_length',
 'forecaster_X__realdpi__RecursiveTabularRegressionForecaster__


### Advantages of sequential pipelines
* Constructing simple pipelines is very easy.
* Inverse operations are automatically applied.
* This is a mature feature compared to the experimental graphical pipeline.


### Advantages of graphical pipelines
* Enable an easy implementation of complex pipelines
    * By nesting sequential pipelines, even a simplified version of the graphical pipeline is very complicat to implement.
    * By nesting sequential pipelines, some graphical pipelines are not possible to implement (e.g., the example with coupled ForecastX).
* The parameter structure is simpler compared to sequential pipelines.
    * Thus easier to tune the structure also in complex secnarios. How would you tune the edges of sequential pipelines?
* Only one estimator to track compared to multiples in the sequential pipeline example.



---

### Credits: notebook 4

notebook creation: benheid

based on:

* pyData Prague 2023 notebook (benheid, fkiraly)

graphical pipeline: benheid, fkiraly, pywatts team\
forecaster pipelines: fkiraly, aiwalter\
transformer pipelines & compositors: fkiraly, mloning, miraep8\
dunder interface: fkiraly, miraep8\